# gnucash-historical-price-data
_**gnucash:**_ v3.8
_**format:**_ XML, XML Compressed  

A script to patch existing gnucash ledgers with stock price data (from the internet).

Script assumes the following:
   
 - currency commodities already exist in book
 - security commodities already exist in book with correct exchange
 - uniform denominations for all equities (i.e 1/10,000)

In [7]:
from copy import copy
from datetime import datetime, date, timedelta
import gzip
from urllib.parse import urlencode
import uuid

from bs4 import BeautifulSoup, FeatureNotFound
import pandas as pd

## Configuration:

In [8]:

book = f'../Financial/michael-2022-2023.gnucash'

target = book
# target = f'test-with-history{EXT}'

start_date = datetime.strptime('2019-01-01', '%Y-%m-%d')
end_date = datetime.today() - timedelta(days=9)
interval = '1d' ## one of ['1d', '5d', '1wk', '1mo', '3mo']

unit_sz = 1000000

root_url = 'https://query1.finance.yahoo.com/v7/finance/download/'
portfolio = []

## Patch:

In [9]:
compressed = True

try:
    with gzip.open(book, 'rb') as buff:
        book = BeautifulSoup(buff.read(), 'lxml-xml')
        compressed = True
except gzip.BadGzipFile as bgf:
    try:
        with open(book, 'rb') as buff:
            book = BeautifulSoup(buff.read(), 'lxml-xml')
    except Exception as exc:
        raise exc

pricedb = book.find('gnc:pricedb')

start_ts = int(datetime.timestamp(start_date))
end_ts = int(datetime.timestamp(end_date))

payload = {
    'period1': start_ts,
    'period2': end_ts,
    'interval': interval,
    'events': 'history'
}

assert payload['interval'] in ('1d', '5d', '1wk', '1mo', '3mo'), 'Invalid interval'
assert pricedb.get('version') == '1', 'Unsupported Version'
assert payload['period1'] < payload['period2'], '\'End\' date must be before \'Start\' date'
assert payload['period2'] < datetime.timestamp(datetime.today()), '\'End\' date must be in the past'

In [10]:
hist = []
template = pricedb.price
for stock in portfolio:
    data = pd.read_csv(f'{root_url}{stock["SYM"]}?{urlencode(payload)}')
    
    for date, price in zip(data["Date"], data["Close"]):
        record = copy(template)  
        record.find('price:id').string = str(uuid.uuid4()).replace('-', '')
        record.find('cmdty:id').string = stock['SYM']
        record.find('cmdty:space').string = stock['XCHG']
        record.find('ts:date').string = f'{date} 17:00:00 +0000'
        record.find('price:value').string = f'{int(price * unit_sz)}/{unit_sz}'
   
        hist.append(record)

old_data = [datum.extract() for datum in pricedb.findAll('price')]

for entry in hist:
    pricedb.append(entry)

book.findAll('gnc:count-data', {'cd:type' : 'price'}).string = len(hist)
assert len([item for item in pricedb.find_all('price')]) == len(hist)    

if compressed:
    with gzip.open(target, 'w') as buff:
        buff.write(bytes(str(book), 'UTF-8'))
else:
    with open(target, "w") as buff:
        buff.write(str(book))
print("Patch completed.")

Patch completed.


In [11]:
'foo'

'foo'